Este cuaderno va a estar dedicado de manera exclusiva a la generacion de una practica conocida como "Data Augmentation".

Esta practica permite generar nuevos datos para entrenar a partir de unos nuevos. Como disponemos de imagenes para el entrenamiento, utilizare una libreria de Keras para la generacion de nuevas imagenes.

El objetivo primordial de este proceso es lograr obtener un numero similar (o identico) de ejemplos para cada uno de las
etiquetas que el modelo debera aprender a clasificar.

In [ ]:
# Para empezar,reviso los ejemplos que tengo para cada una de las etiquetas

import os

directorioImagenes = 'datasets_animales/'

for carpeta in os.listdir(directorioImagenes):
    print('Existen un total de {} imagenes para la especie {}.'.format(len(os.listdir(os.path.join(directorioImagenes, carpeta))),
                                                                       carpeta))

In [ ]:
# Teniendo en cuenta que la etiqueta que mas ejemplos tiene es la de dog (perro), me basare en este numero para crear]
# nuevas copias del resto.

LIMITE_SUPERIOR = len(os.listdir(os.path.join(directorioImagenes, 'dog')))

def asignarImagenesEspecie():
    
    """Funcion sin parametros que se crear un diccionario cuyos pares clave-valor tienen la siguiente estructura
       ==> especieAnimal: nImagenesFaltantes_limiteSuperior.
       
       Dicho diccionario es creado iterando sobre el contenido del directorio que contiene las imagenes, y finalmente
       es devuelto por la funcion al entorno global del cuaderno."""
    
    dictImagenesAnimales = {}
    
    for carpeta in os.listdir(directorioImagenes):
        
        if carpeta == 'dog':  # Como dog tiene el numero maximo de imagenes, no generamos imagenes para esta etiqueta.
            continue
        
        dictImagenesAnimales[carpeta] = len(os.listdir(os.path.join(directorioImagenes, carpeta)))
        
        
    return dictImagenesAnimales


dictImagenesAnimales = asignarImagenesEspecie()

In [ ]:
# Reviso las imagenes que necesitamos para cada una de las etiquetas

for clave, valor in dictImagenesAnimales.items():
    
    print(clave, '==>', valor)

In [ ]:
# Para llevar a cobo el aumento de datos, hago uso de una clase del framework Keras.

from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import cv2
import numpy as np
from multiprocessing import Pool

In [ ]:
# Como vemos, tenemos un numero diferente de ejemplos para cada etiqueta, lo que puede causar en el modelo una cierta tendencia
# a clasificar falsos positivos. Tras el entrenamiento, representaremos el rendimiento del modelo en una matriz de confusion.

# RECORDATORIO ==> Es importante instalar las dependencias incluidas en el documento requirements.txt .

################################################################################################################################

# Directorio de entrada y salida
# directorioImagenes = 'datasets_animales/'
output_directory = 'datasets_animales/'  

# Creo un generador de data augmentation
datagen = ImageDataGenerator(rotation_range= 30, horizontal_flip=True, fill_mode='nearest')


# Recorrer cada imagen en el dataset original y obtener su ruta
for carpeta in tqdm(os.listdir(directorioImagenes)):
    
    if carpeta in dictImagenesAnimales.keys():
        
        if len(os.listdir(os.path.join(directorioImagenes, carpeta))) >= LIMITE_SUPERIOR:
            
            print('La etiqueta {} ya dispone de la cantidad de ejemplos deseada'.format(carpeta))
            continue
        
        else:
        
            imagenesNuevas = 0

            for image_file in os.listdir(os.path.join(directorioImagenes, carpeta)):

                if imagenesNuevas >= (LIMITE_SUPERIOR - dictImagenesAnimales[carpeta]):
                    break
                else:

                    imagenesNuevas +=1 

                    image = cv2.imread(os.path.join(directorioImagenes + carpeta + '/', image_file))

                    # Verifica si la imagen se ha cargado correctamente
                    if image is not None:
                        # Agrega una dimensión al array de la imagen
                        image = np.expand_dims(image, axis=0)
                        # Genera imágenes aumentadas y guárdalas en el directorio de salida
                        i = 0
                        for batch in datagen.flow(image, batch_size=1, save_to_dir=output_directory + carpeta + '/', save_format='jpg'):
                            i += 1
                            if i == 1:
                                break
                    else:
                        print('Error al cargar la imagen: {}'.format(image_file))

            print('Se han generado {} nuevas imagenes para la etiqueta {}'.format(imagenesNuevas, carpeta))

print('Data augmentation completada.')
print('Las imágenes generadas han sido alojadas en el directorio {}.'.format(output_directory))

In [ ]:
# Aparentemente, hemos finalizado el proceso de generacion de imagenes (seria buen idea almacenar unas pocas en una carpeta 
# y compararlas con las originales.)

# Voy a mostrar el numero de elementos dentro de cada uno de los subdirectorios correspondientes a cada etiqueta.

for carpeta in os.listdir(directorioImagenes):
    
    print('Existen {} ejemplos para la etiqueta "{}".'.format(len(os.listdir(os.path.join(directorioImagenes, carpeta))), 
                                                              carpeta))

In [ ]:
# Al revisar, veo que me he pasado generando las imagenes, por lo que voy a eliminar ejemplos de todas las etiquetas
# que tengan una cantidad de ejemplos superior al valor de la constante 'LIMITE_SUPERIOR'.


for carpeta in os.listdir(directorioImagenes):

    numero_imagenes_a_eliminar = len(os.listdir(os.path.join(directorioImagenes, carpeta))) - LIMITE_SUPERIOR
    imagenesEliminadas = 0
    print('Se va a proceder a eliminar {} imagenes correspondientes a la etiqueta {}.'.format(numero_imagenes_a_eliminar, 
                                                                                              carpeta))

    # Lista de ejemplos en el subdirectorio de la etiqueta iterada
    archivos = os.listdir(os.path.join(directorioImagenes, carpeta))

    # Si hay suficientes imágenes en el directorio
    if len(archivos) > numero_imagenes_a_eliminar:
        # Ordena la lista de archivos por fecha de modificación (más recientes primero)
        archivos.sort(key=lambda x: os.path.getmtime(os.path.join(directorioImagenes + carpeta + '/', x)), reverse=True)

        # Elimina las últimas 'numero_imagenes_a_eliminar' imágenes
        for i in tqdm(range(numero_imagenes_a_eliminar)):
            archivo_a_eliminar = archivos[i]
            ruta_completa = os.path.join(directorioImagenes + carpeta + '/', archivo_a_eliminar)
            try:
                os.remove(ruta_completa)
                imagenesEliminadas +=1
                #print(f'Imagen eliminada: {archivo_a_eliminar}')
            except Exception as e:
                print(f'Error al eliminar la imagen {archivo_a_eliminar}: {e}')
        print('Se han eliminado correctamente {} imagenes para la etiqueta {}.'.format(imagenesEliminadas, carpeta))
    else:
        print('No hay suficientes imágenes en el directorio para eliminar.')

In [ ]:
# Tras esto, reviso la cantidad de ejemplos de los que dispongo finalmente para cada imagen.

for carpeta in os.listdir(directorioImagenes):
    
    print('Existen {} ejemplos para la etiqueta {}.'.format(len(os.listdir(os.path.join(directorioImagenes, carpeta))),
                                                            carpeta))